In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sim.dy.model_intv import ModelIntv
import sim.dy.keys as I
import numpy.random as rd

In [2]:
n_collect = 5

In [3]:
def get_stats(ms, idx, **kwargs):
    ms_stats = dict()
    ms_stats.update(kwargs)
    ms_stats['Pop0'] = ms.Pop.iloc[0]
    
    for key in idx[1:]:
        try:
            ms_stats[key] = (ms[key] * ms.Pop).sum() * 0.5
        except KeyError:
            ms_stats[key] = 0
    return ms_stats

In [4]:
def collect_fux(pars_post, n_collect=20, coverage=0.03, duration=2, tp="VulACF"):    
    m = ModelIntv()
    
    sel = rd.choice(list(range(len(pars_post))), n_collect)
    sel.sort()

    mss = list()
    mss_stats = list()


    for i in sel:
        pars = pars_post[i]
        y1, pars = m.find_baseline(pars, 2022)
    
        _, ms, _ = m.simulate_onward(y1, pars, intv={}, t_end=2031)
        idx = ['Pop', 'IncR', 'MorR'] + [c for c in ms.columns if c.startswith('ACF_Vul')]
        ms = ms.reset_index().assign(Key=i, FollowUp=0, Duration=0)
        mss.append(ms)
        ms_stats = get_stats(ms, idx, Key=i, Scenario='Baseline', Coverage=coverage, FollowUp=0, Duration=0)
        mss_stats.append(ms_stats)
    
#         for n_fu in np.linspace(0, 6, 5): # + [24, 30, 36]:
#             if n_fu > 0:
#                 fu = 1 / n_fu
#             else:
#                 fu = 0

        n_fu, fu = 2, 0.5
        for duration in [0, 1, 2, 3]:
            for c1 in np.linspace(0, 1, 10):
                intv = {'VulACF': {'Coverage': c1, 'FollowUp': fu, 'Duration': duration}}

                _, ms, _ = m.simulate_onward(y1, pars, intv=intv, t_end=2031)
                try:
                    ms = ms.reset_index().assign(Key = i, FollowUp=fu, Duration=duration)
                    idx = ['Pop', 'IncR', 'MorR'] + [c for c in ms.columns if c.startswith('ACF_Vul')]
                    mss.append(ms)
                    ms_stats = get_stats(ms, idx, Key=i, Scenario='Intervention', Coverage=c1, FollowUp=fu, Duration=duration)
                    mss_stats.append(ms_stats)
                except TypeError:
                    pass
            
    mss = pd.concat(mss).fillna(0)
    mss_stats = pd.DataFrame(mss_stats).fillna(0)
    return mss, mss_stats

In [5]:
for folder in ['dy_hi']:
    print(folder)
    out_path = f'out/{folder}'
    pars_post = json.load(open(f'{out_path}/Post.json', 'r'))
    r_acf = json.load(open(f'{out_path}/R_ACF_Bg.json', 'r'))

    for p in pars_post:
        p.update(r_acf)
        
    mss, mss_stats = collect_fux(pars_post, n_collect=n_collect, coverage=0.03)
    mss.to_csv(f'{out_path}/Sim_VulACF_cex.csv')
    mss_stats.to_csv(f'{out_path}/Sim_VulACF_cex_stats.csv')

dy_hi


In [6]:
mss_stats

,Key,Scenario,Coverage,FollowUp,Duration,Pop0,IncR,MorR,ACF_Vul_Footfall,ACF_Vul_Screened,...,ACF_VulFu_Confirmed,ACF_VulFu_Sym,ACF_VulFu_Vul,ACF_VulFu_CXR,ACF_VulFu_Xpert,ACF_VulFu_Yield,ACF_VulFu_TP,ACF_VulFu_DS_Fl,ACF_VulFu_DR_Fl,ACF_VulFu_DR_Sl
0,33,Baseline,0.030000,0.0,0,2.383080,0.113892,0.015285,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
1,33,Intervention,0.000000,0.5,0,2.383080,0.113892,0.015285,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
2,33,Intervention,0.111111,0.5,0,2.383080,0.107088,0.013942,2.452216,0.679121,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3,33,Intervention,0.222222,0.5,0,2.383080,0.100952,0.012763,4.904468,1.357823,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
4,33,Intervention,0.333333,0.5,0,2.383080,0.095731,0.011774,7.356761,2.036197,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,1461,Intervention,0.555556,0.5,3,2.383155,0.071344,0.007761,12.258769,3.288614,...,0.437891,12.244065,0.0,0.0,0.437891,0.009507,0.002833,0.002724,0.0,0.000109
201,1461,Intervention,0.666667,0.5,3,2.383155,0.069022,0.007360,14.710597,3.945831,...,0.476470,14.692878,0.0,0.0,0.476470,0.010115,0.002850,0.002740,0.0,0.000110
202,1461,Intervention,0.777778,0.5,3,2.383155,0.067032,0.007017,17.162464,4.602950,...,0.508256,17.141691,0.0,0.0,0.508256,0.010578,0.002824,0.002715,0.0,0.000109
203,1461,Intervention,0.888889,0.5,3,2.383155,0.065306,0.006721,19.614371,5.259985,...,0.534888,19.590504,0.0,0.0,0.534888,0.010943,0.002779,0.002672,0.0,0.000107
